In [8]:
import pandas as pd
import glob
import math
from matplotlib import pyplot as plt
import numpy as np
import csv
import time
import gc
import scipy.stats
from scipy.stats import kurtosis
from scipy.stats import skew
from scipy.stats import iqr
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks
from scipy.stats import entropy
import statsmodels.api as sm
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from pandas import read_csv
from pandas.plotting import lag_plot
from pandas.plotting import autocorrelation_plot
from statsmodels.graphics.tsaplots import plot_acf
#import pywt
from scipy import signal
import matplotlib.pyplot as plt
from scipy import special
from pathlib import Path
import re

# Import training data

In [9]:
path_input: str = "./train" # input path
sensor: int = 5 # input sensor

In [10]:
if not isinstance(path_input, str) or not (path:=Path(path_input)).is_dir():
    raise ValueError(f"Input {path_input} is not a valid directory!")

all_files = glob.glob(path.as_posix() + "/*.csv")

if len(all_files) == 0:
    raise Exception("No files. Abort.")

print(f"Found {len(all_files)} files!")

Found 14 files!


# Prepare training data

In [11]:
np.random.seed(1234)

# Segmentieren der Zeitreihen

# Loesche ab Begin_Del 1769 / 710
Begin_Del = 710
# Loesche die ersten Teil
To_Del = 20

Num_Pro = 139


# Laden der Trainingsdaten
train_data = pd.read_csv(all_files[sensor - 1], decimal=",", sep=";", header=0)

# Segmentieren der Daten

train_data = train_data.drop(train_data.index[Begin_Del : len(train_data)])
train_data = train_data.drop(train_data.index[0:To_Del])

# Datenformat float
train_data = train_data.to_numpy().astype("float64")

# Variable for time entries
time = train_data[:, 0]

# delete nan columns
train_data = train_data[:, ~np.isnan(train_data).any(axis=0)]

# delete first column (time entries)
train_data = train_data[:, 1 : Num_Pro + 1]

# Convert
train_data[train_data == -2.8026e-45] = -0.0001
train_data[train_data == 2.8026e-45] = 0.0001

train_data[train_data == -2.8e-41] = -0.0001
train_data[train_data == 2.8e-41] = 0.0001

## Define X_train and X_test

X_train = train_data[:, 0:20]
X_test = train_data[:, 115]
X_test = X_test.transpose().reshape(-1, 1)

C:\Users\gerri\AppData\Local\Temp\ipykernel_9572\1875244818.py:14: DtypeWarning: Columns (7,9,13,23,26,32,37,47,50,55,83,87,91,105,108,117,127,129,138,149,151,155,165,168,174,179,189,192,197,225,229,233,247,250,259,269,271,280,291,293,297,307,310,316,321,331,334,339,367,371,375,389,392,401,411,413,422,433,435,439,449,452,458,463,473,476,481,509,513,517,531,534,543,553,555,564,575,577,581,591,594,600,605,615,618,623,651,655,659,673,676,685,695,697,706,717,719,723,733,736,742,747,757,760,765,793,797,801,815,818,827,837,839,848,859,861,865,875,878,884,889,899,902,907,935,939,943,957,960,969,979,981,990,1001,1003,1007,1017,1020,1026,1031,1041,1044,1049,1077,1081,1085,1099,1102,1111,1121,1123,1132,1143,1145,1149,1159,1162,1168,1173,1183,1186,1191,1219,1223,1227,1241,1244,1253,1263,1265,1274,1285,1287,1291,1301,1304,1310,1315,1325,1328,1333,1361,1365,1369,1383,1386,1395,1405,1407,1416,1427,1429,1433,1443,1446,1452,1457,1467,1470,1475,1503,1507,1511,1525,1528,1537,1547,1549,1558,1569,1571,157

# Train & Test

## Create configuration for training & testing

In [12]:
from basys4ipps_ifw_agent.Agent.basys_agent import BasysAgent, BasysConfig

basys_config = BasysConfig()
basys_config.alpha_safety_factor = 0.999

In [13]:
# Ende Daten laden ######################################

# initialise model
# baseline_model = unified_outlier_score(base_model = "KNN", outlier_score_scaling = "gaussian", alpha_safety_factor = 0.99999, feature_group = "general_purpose", feature_scaling_method = "standardize", learning_type = "static", machine_component = "axis_drive", segementation_start = 20, segmentation_end = 710)

# fit the model on train data
# baseline_model.fit(X_train)

# predict using baseline model
# score_prob, alarm, exp_downtime = baseline_model.predict(X_test)


agent = BasysAgent(basys_config)

print("Start training")
agent.fit(X_train)

print("Start testing")
score_prob, alarm, exp_downtime = agent.predict(X_test)


Start training
{'contamination': 0.1, 'n_neighbors': 5, 'method': 'largest', 'radius': 1.0, 'algorithm': 'auto', 'leaf_size': 30, 'metric': 'minkowski', 'p': 2, 'metric_params': None, 'n_jobs': 1, 'neigh_': NearestNeighbors(n_jobs=1), '_classes': 2, 'tree_': <sklearn.neighbors._kd_tree.KDTree object at 0x00000196633AFD30>, 'decision_scores_': array([7.22462258, 2.92445374, 4.04769915, 3.62095359, 4.59976424,
       3.97825938, 3.62095359, 2.65778377, 1.77502885, 1.91802469,
       1.47655424, 0.        , 1.91671737, 1.84873355, 0.        ,
       0.        , 0.        , 1.6667368 , 0.        , 0.        ]), 'threshold_': 4.1029056576726415, 'labels_': array([1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]), '_mu': 2.163814277115093, '_sigma': 1.90577831155715}
Start testing


ValueError: not enough values to unpack (expected 3, got 2)